# 

# Generating synthetic data
- This jupyter note book will run through set up and execution for generating synthetic data

In [ ]:
# Setting up containers
!cd ../umccr-dictionary && make down
!cd ../umccr-dictionary && make pull
!cd ../umccr-dictionary && make up
!cd ../umccr-dictionary && make ps
!cd ../umccr-dictionary && make restart

# Generating synthetic data using umccr-dictionary
- This part of the script assumes that bundled json has been created using:

```bash
!cd ../umccr-dictionary && make compile program=schema_dev
```

- If this has not yet been created, make sure to create it first using the [schema_dev_framework.ipynb notebook](./schema_dev_framework.ipynb)


In [ ]:
# Generating synthetic data using umccr-dictionary
!cd ../umccr-dictionary && make simulate program=schema_dev project=tmp_project max_samples=110

In [3]:
# Copying synthetic data back to gen3schemadev
!mkdir -p ../output/synthetic_data/raw_umccr
!mkdir -p ../output/synthetic_data/raw_gen3schemadev
!mkdir -p ../output/synthetic_data/transformed
!cp -r ../umccr-dictionary/data/schema_dev/ ../output/synthetic_data/raw_umccr
# cleaning temp data
!test -d ../umccr-dictionary/data/schema_dev/ && rm -r ../umccr-dictionary/data/schema_dev/

# Running plausible data generation
- Make sure to specify gurl, which reads the SD and mean values from a google sheet and generates a synthetic data set
- Refer to [plausible_values gsheet](https://docs.google.com/spreadsheets/d/1zjDBDvXgb0ydswFBwy47r2c8V1TFnpUj1jcG0xsY7ZI/edit#gid=1771305859)

In [ ]:
# Running plausible data generation
!(cd .. && bash scripts/add_plausible_values.sh -s tmp_project -i output/synthetic_data/raw_umccr -g 'https://docs.google.com/spreadsheets/d/1zjDBDvXgb0ydswFBwy47r2c8V1TFnpUj1jcG0xsY7ZI/edit#gid=1771305859' -o output/synthetic_data/raw_gen3schemadev)

# Update metadata with dummy file names
- Derived from these [dummy files](../file_type_templates/)
- You can read more about how this step works here: [file_type_templates/README.md](../file_type_templates/README.md)

In [ ]:
import gen3schemadev

project_id_list = ['tmp_project']
templates_path = '../file_type_templates'
json_data_path = f'../output/synthetic_data/raw_gen3schemadev/'
output_dir = f'../output/synthetic_data/raw_gen3schemadev_updated_fn/'

synthdata = gen3schemadev.gen3synthdata.gen3SynthFiles(json_data_path, project_id_list, templates_path, output_dir)
synthdata.copy_metadata_files()
synthdata.update_fn_metadata()

Synthetic data files have now been generated, and the metadata has been updated with dummy file names.
- You can find the data in `../output/synthetic_data/raw_gen3schemadev_updated_fn/tmp_project`